In [2]:
import tensorflow as tf

In [3]:
# First abstraction, in tf everything is a graph
two_node = tf.constant(2)
print (two_node)

Tensor("Const:0", shape=(), dtype=int32)


In [4]:

#In tensorflow the variable are pointers
third_node = two_node

print(two_node)
print(third_node)

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const:0", shape=(), dtype=int32)


In [5]:
# everything done in tf will evaluate to a node in a graph
sum_node = third_node + two_node
print(sum_node)

Tensor("add:0", shape=(), dtype=int32)


In [6]:
# Second abstraction, Session
# You need a session in order to evaluate the graph

sess = tf.Session()

In [7]:
print(sess.run(sum_node))
print(sess.run([sum_node, two_node, third_node]))

4
[4, 2, 2]


In [8]:
# placeholders allows you pass value when running
# the session as a dict
ph01 = tf.placeholder( tf.int32)
print(sess.run(ph01, feed_dict={ph01: 2}))

2


In [21]:
# Third abstraction, computation path
import tensorflow as tf
input_placeholder = tf.placeholder(tf.int32)
three_node = tf.constant(3)
sum_node = input_placeholder + three_node
print (sess.run(three_node))
print (sess.run(sum_node, feed_dict={input_placeholder: 5}))

3
8


In [23]:
# variables and side effects
# pass the name and the shape
tf.reset_default_graph()
count_variable  = tf.get_variable("count", [])
sess = tf.Session()

# assing value to variable
zero_node = tf.constant(0.)
assign_node = tf.assign( count_variable, zero_node)
sess.run( assign_node )

print(sess.run(count_variable))

0.0


<img src="tensorflow_test_src/fig13.png",width=200,height=200>

In [31]:
# initialiazer for variables
tf.reset_default_graph()
const_init_node = tf.constant_initializer(0.)
count_variable = tf.get_variable("count01", [], initializer=const_init_node)

# update the variable reference to be use by the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

print (sess.run(count_variable))

0.0


<img src="tensorflow_test_src/fig14.png",width=200,height=200>

** note **
You may encounter Tensorflow code with variable sharing, which involves creating a scope and setting “reuse=True”. I strongly recommend that you don’t use this in your own code. If you want to use a single variable in multiple places, simply keep track of your pointer to that variable’s node programmatically, and re-use it when you need to. In other words, you should have only a single call of tf.get_variable() for each parameter you intend to store in memory.

## In deep learning you usually do 

1. Get an input and true_output
2. Compute a “guess” based on the input and your parameters
3. Compute a “loss” based on the difference between your guess and the true_output
4. Update the parameters according to the gradient of the loss

### Lets look at a linear regression in tf

```python
import tensorflow as tf
tf.reset_default_graph()
### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
y_guess = m * x + b

loss = tf.square(y - y_guess)

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(100000):
  ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b

  ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _ = sess.run([loss, train_op], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
  
    if update_i % 10000 == 0:
        print (update_i, _loss)

# print last loss 
print (update_i, _loss)

### finally, print out the values we learned for our two variables
print ("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print ("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))
```

In [37]:
import tensorflow as tf
tf.reset_default_graph()
### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
y_guess = m * x + b

loss = tf.square(y - y_guess)

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(100000):
  ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b

  ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _ = sess.run([loss, train_op], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
  
    if update_i % 10000 == 0:
        print (update_i, _loss)

# print last loss 
print (update_i, _loss)

### finally, print out the values we learned for our two variables
print ("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print ("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))

0 1.02288
10000 0.000746677
20000 0.000103315
30000 1.3175e-05
40000 3.85593e-07
50000 1.22382e-08
60000 3.32898e-09
70000 7.78186e-11
80000 4.60432e-12
90000 6.00409e-11
99999 2.75122e-11
True parameters:     m=0.9632, b=0.9507
Learned parameters:  m=0.9632, b=0.9507


## Debugging with tf.Print

In [50]:
tf.reset_default_graph()

two_node = tf.constant(2)
three_node = tf.constant(3)
sum_node = two_node + three_node
print_sum_node = tf.Print(sum_node, [two_node, three_node])

sess = tf.Session()

print(sess.run(print_sum_node))

5


<img src="tensorflow_test_src/fig15.png",width=200,height=200>